<a href="https://colab.research.google.com/github/rajitakolla/computational_methods/blob/master/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [2019 Dell labtop](https://www.amazon.com/Dell-Inspiron-5000-5570-Laptop/dp/B07N49F51N/ref=sr_1_11?crid=1IJ7UWF2F4GHH&keywords=dell%2Bxps%2B15&qid=1580173569&sprefix=dell%2Caps%2C181&sr=8-11&th=1) on amazon.

(2) Collect the top 100 User Reviews of the film [Joker](https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv) from IMDB.

(3) Collect the abstracts of the top 100 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 100 tweets by using hashtag ["#CovidVaccine"](https://twitter.com/hashtag/CovidVaccine) from Twitter. 


In [112]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chro

In [113]:
# Write your code here
import urllib.request
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)


In [114]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [115]:
def fetch_data(driver, class_name_val, css_selector_val):
  result = list()
  values = driver.find_elements_by_class_name(class_name_val)
  for value in values:
    result.append(value.find_element_by_css_selector(css_selector_val).text)
  return result
  pass

In [116]:
import time
driver.get("https://www.imdb.com/title/tt7286456/reviews?sort=userRating&dir=desc&ratingFilter=0")
#activate the button load_more and run the loop 4 times so to make sure we have 100 words
i=0
while(i<3):
  button = driver.find_element_by_xpath('//*[@id ="load-more-trigger"]').click()
  i = i+1
  time.sleep(2)

reviews = fetch_data(driver, "content", ".text.show-more__control")
user_names = fetch_data(driver, "display-name-date","a")

In [117]:
#create a data frame of usernames and reviews
required_df = pd.concat([pd.DataFrame(user_names,columns=['User Names']),pd.DataFrame(reviews,columns=['Reviews'])],axis=1)
required_df

,User Names,Reviews
0,morino-59166,When I saw the magnificent Heath ledger a few ...
1,asassandroo,I can say that i'm really grateful that movies...
2,samanehdds,I'm not a fan of Batman movies but went to see...
3,guillevica,"What a master piece, it so so wondeful, all in..."
4,ahurworth,Has to be the best movie I've seen in years. I...
...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...
96,afnansaeed-76293,One of the best movies. I really recommend it ...
97,legorocks-30147,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,..."


In [118]:
required_df.to_csv("output1.csv")

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [119]:
#removing noise and special characters
import string
punctuations = required_df['Reviews'].apply(lambda x:''.join([i for i in x if i not in string.punctuation])).to_frame().rename(columns={"Reviews":"Punctuations"})

punctuations_and_special_characters_df = punctuations['Punctuations'].apply(lambda x:''.join([i for i in x if i.isalnum() or i == " "])).to_frame().rename(columns={"Punctuations":"Punctuations and special characters"})

required_df_1 = pd.concat([required_df,punctuations_and_special_characters_df],axis=1)
required_df_1

,User Names,Reviews,Punctuations and special characters
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...
...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...
97,legorocks-30147,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...


In [120]:
#removing numbers
numbers_df = required_df_1['Punctuations and special characters'].apply(lambda x:''.join([i for i in x if i == " " or i.isalpha()])).to_frame().rename(columns={"Punctuations and special characters":"numbers"})
required_df_2 = pd.concat([required_df_1,numbers_df],axis=1)
required_df_2

,User Names,Reviews,Punctuations and special characters,numbers
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...,I can say that im really grateful that movies ...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...,Im not a fan of Batman movies but went to see ...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...,What a master piece it so so wondeful all in t...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...,Has to be the best movie Ive seen in years It ...
...,...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns this...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...,One of the best movies I really recommend it t...
97,legorocks-30147,,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...,This movie is truly a masterpiece The acting c...


In [121]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [122]:
list_of_words = set(stopwords.words('english')) 

In [123]:
#Removing stop words

stop_words_df =  required_df_2['numbers'].apply(lambda x:' '.join([i for i in x.split() if i not in list_of_words])).to_frame().rename(columns={"numbers":"stop_words"})
result_df_3 = pd.concat([required_df_2,stop_words_df],axis=1)
result_df_3

,User Names,Reviews,Punctuations and special characters,numbers,stop_words
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw magnificent Heath ledger years go e...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...,I can say that im really grateful that movies ...,I say im really grateful movies like one haven...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...,Im not a fan of Batman movies but went to see ...,Im fan Batman movies went see teenage son Its ...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...,What a master piece it so so wondeful all in t...,What master piece wondeful movie great constru...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...,Has to be the best movie Ive seen in years It ...,Has best movie Ive seen years It feel Taxi Dri...
...,...,...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns role...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...,One of the best movies I really recommend it t...,One best movies I really recommend gys
97,legorocks-30147,,,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...,This movie is truly a masterpiece The acting c...,This movie truly masterpiece The acting cenima...


In [124]:
#lower case all the texts


lower_case_df =  result_df_3['stop_words'].apply(lambda x: x.lower()).to_frame().rename(columns={"stop_words":"lower_case"})
result_df_4 = pd.concat([result_df_3,lower_case_df],axis=1)
result_df_4

,User Names,Reviews,Punctuations and special characters,numbers,stop_words,lower_case
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw magnificent Heath ledger years go e...,when i saw magnificent heath ledger years go e...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...,I can say that im really grateful that movies ...,I say im really grateful movies like one haven...,i say im really grateful movies like one haven...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...,Im not a fan of Batman movies but went to see ...,Im fan Batman movies went see teenage son Its ...,im fan batman movies went see teenage son its ...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...,What a master piece it so so wondeful all in t...,What master piece wondeful movie great constru...,what master piece wondeful movie great constru...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...,Has to be the best movie Ive seen in years It ...,Has best movie Ive seen years It feel Taxi Dri...,has best movie ive seen years it feel taxi dri...
...,...,...,...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns role...,absolutely great performance phoenix owns role...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...,One of the best movies I really recommend it t...,One best movies I really recommend gys,one best movies i really recommend gys
97,legorocks-30147,,,,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...,This movie is truly a masterpiece The acting c...,This movie truly masterpiece The acting cenima...,this movie truly masterpiece the acting cenima...


In [135]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [108]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
   
ps = PorterStemmer()

In [143]:
#stemming
tokenisation = result_df_4["lower_case"].apply(lambda x : word_tokenize(str(x))).to_frame().rename(columns = {"lower_case":"tokenisation"})
stemming_df = tokenisation['tokenisation'].apply(lambda x: ' '.join([ps.stem(i) for i in x])).to_frame().rename(columns={"tokenisation":"stemming"})
result_df_5 = pd.concat([result_df_4,stemming_df],axis=1)
result_df_5

,User Names,Reviews,Punctuations and special characters,numbers,stop_words,lower_case,stemming
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw magnificent Heath ledger years go e...,when i saw magnificent heath ledger years go e...,when i saw magnific heath ledger year go excel...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...,I can say that im really grateful that movies ...,I say im really grateful movies like one haven...,i say im really grateful movies like one haven...,i say im realli grate movi like one havent die...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...,Im not a fan of Batman movies but went to see ...,Im fan Batman movies went see teenage son Its ...,im fan batman movies went see teenage son its ...,im fan batman movi went see teenag son it trul...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...,What a master piece it so so wondeful all in t...,What master piece wondeful movie great constru...,what master piece wondeful movie great constru...,what master piec wond movi great construct mov...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...,Has to be the best movie Ive seen in years It ...,Has best movie Ive seen years It feel Taxi Dri...,has best movie ive seen years it feel taxi dri...,ha best movi ive seen year it feel taxi driver...
...,...,...,...,...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns role...,absolutely great performance phoenix owns role...,absolut great perform phoenix own role the cha...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...,One of the best movies I really recommend it t...,One best movies I really recommend gys,one best movies i really recommend gys,one best movi i realli recommend gy
97,legorocks-30147,,,,,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...,This movie is truly a masterpiece The acting c...,This movie truly masterpiece The acting cenima...,this movie truly masterpiece the acting cenima...,thi movi truli masterpiec the act cenimatograp...


In [139]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [147]:
#lemmetization

import textblob
lemmetization_df = tokenisation['tokenisation'].apply(lambda x: ' '.join([textblob.Word(i).lemmatize() for i in x])).to_frame().rename(columns={"tokenisation":"lemmitization"})
filtered_data_df = pd.concat([result_df_5,lemmetization_df],axis=1)
filtered_data_df

,User Names,Reviews,Punctuations and special characters,numbers,stop_words,lower_case,stemming,lemmitization
0,morino-59166,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw the magnificent Heath ledger a few ...,When I saw magnificent Heath ledger years go e...,when i saw magnificent heath ledger years go e...,when i saw magnific heath ledger year go excel...,when i saw magnificent heath ledger year go ex...
1,asassandroo,I can say that i'm really grateful that movies...,I can say that im really grateful that movies ...,I can say that im really grateful that movies ...,I say im really grateful movies like one haven...,i say im really grateful movies like one haven...,i say im realli grate movi like one havent die...,i say im really grateful movie like one havent...
2,samanehdds,I'm not a fan of Batman movies but went to see...,Im not a fan of Batman movies but went to see ...,Im not a fan of Batman movies but went to see ...,Im fan Batman movies went see teenage son Its ...,im fan batman movies went see teenage son its ...,im fan batman movi went see teenag son it trul...,im fan batman movie went see teenage son it tr...
3,guillevica,"What a master piece, it so so wondeful, all in...",What a master piece it so so wondeful all in t...,What a master piece it so so wondeful all in t...,What master piece wondeful movie great constru...,what master piece wondeful movie great constru...,what master piec wond movi great construct mov...,what master piece wondeful movie great constru...
4,ahurworth,Has to be the best movie I've seen in years. I...,Has to be the best movie Ive seen in years It ...,Has to be the best movie Ive seen in years It ...,Has best movie Ive seen years It feel Taxi Dri...,has best movie ive seen years it feel taxi dri...,ha best movi ive seen year it feel taxi driver...,ha best movie ive seen year it feel taxi drive...
...,...,...,...,...,...,...,...,...
95,shinginharig,Absolutely great performance. Phoenix owns thi...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns this...,Absolutely great performance Phoenix owns role...,absolutely great performance phoenix owns role...,absolut great perform phoenix own role the cha...,absolutely great performance phoenix owns role...
96,afnansaeed-76293,One of the best movies. I really recommend it ...,One of the best movies I really recommend it t...,One of the best movies I really recommend it t...,One best movies I really recommend gys,one best movies i really recommend gys,one best movi i realli recommend gy,one best movie i really recommend gy
97,legorocks-30147,,,,,,,
98,tusharshanker-96210,"This movie is truly a masterpiece. The acting,...",This movie is truly a masterpiece The acting c...,This movie is truly a masterpiece The acting c...,This movie truly masterpiece The acting cenima...,this movie truly masterpiece the acting cenima...,thi movi truli masterpiec the act cenimatograp...,this movie truly masterpiece the acting cenima...


In [149]:
#filtered data output only 3 columns
filtered_data_df[['User Names','Reviews','lemmitization']].to_csv("output2.csv")


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here






**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

In [ ]:
'''
Write your explanations of the constituency parsing tree and dependency parsing tree here



'''

'\nWrite your explanations of the constituency parsing tree and dependency parsing tree here\n\n\n\n'